## Plz consider upvoting and commenting down below about your thoughts about what is good or can be improved. Cheers!

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [4]:
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [5]:
# Analysis
# pip install pandas-profiling

In [6]:
from pandas_profiling import ProfileReport

In [ ]:
# pr = train.profile_report()

In [ ]:
# filling CPU
# pr.to_widgets()

In [ ]:
# Cleaning
!pip install dabl

In [ ]:
import dabl

In [ ]:
train_clean = dabl.clean(train, verbose=0)

In [ ]:
types = dabl.detect_types(train_clean)
print(types) 

In [ ]:
# Not Normally distributed
dabl.plot(train, 'SalePrice')

In [ ]:
# !pip install missingno
import missingno as msno

In [ ]:
msno.matrix(train)

In [ ]:
msno.dendrogram(train)

In [ ]:
msno.bar(train)

In [ ]:
msno.matrix(train.sample(100))

In [ ]:
msno.heatmap(train)

In [ ]:
# !pip install autoviz

In [ ]:
!pip install datasist

In [ ]:
import datasist as ds
ds.structdata.check_train_test_set(train, test, index=None, col=None)

In [ ]:
ds.structdata.describe(train)


In [ ]:
numerical_feats = ds.structdata.get_num_feats(train)
ds.structdata.detect_outliers(train,80,numerical_feats)

In [ ]:
ds.structdata.display_missing(train)['missing_percent'].sort_values(ascending=False)

In [ ]:
all_data, ntrain, ntest = ds.structdata.join_train_and_test(train, test)
print("New size of combined data {}".format(all_data.shape))
print("Old size of train data: {}".format(ntrain))
print("Old size of test data: {}".format(ntest))

#later splitting after transformations
train_new = all_data[:ntrain]
test_new = all_data[ntrain:]

In [ ]:
new_train_df = ds.feature_engineering.drop_missing(train_new,  
                                                    percent=7.0)
# ds.structdata.display_missing(new_train_df)

In [ ]:
ds.structdata.display_missing(new_train_df)['missing_percent'].sort_values(ascending=False)

In [ ]:
ds.feature_engineering.drop_redundant(new_train_df)

In [ ]:
df = ds.feature_engineering.fill_missing_cats(new_train_df)
ds.structdata.display_missing(df)['missing_counts'].sort_values()

In [ ]:
df = ds.feature_engineering.fill_missing_num(new_train_df)
ds.structdata.display_missing(df)

In [ ]:
df = ds.feature_engineering.fill_missing_num(df)
df = ds.feature_engineering.log_transform(df,columns=['Id'])

In [ ]:
df

In [ ]:
features = ['OverallQual' , 'GrLivArea' , 'TotalBsmtSF' , 'BsmtFinSF1' ,
            '2ndFlrSF'    , 'GarageArea', '1stFlrSF'    , 'YearBuilt'  ]

In [ ]:
X_train = df[features]
y_train = df["SalePrice"]
X_test  = test[features]

In [ ]:
X_train      = X_train.fillna(X_train.mean())
X_test       = X_test.fillna(X_test.mean())

In [ ]:
from rgf.sklearn import RGFRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split

estimators =  [('xgb',xgb.XGBRegressor(n_estimators  = 750,learning_rate = 0.02, max_depth = 4)),
               ('cat',CatBoostRegressor(loss_function='RMSE', verbose=False)),
               ('RGF',RGFRegressor(max_leaf=300, algorithm="RGF_Sib", test_interval=100, loss="LS"))]

ensemble = StackingRegressor(estimators      =  estimators,
                             final_estimator =  RandomForestRegressor())

# Fit ensemble using cross-validation
X_tr, X_te, y_tr, y_te = train_test_split(X_train,y_train)
ensemble.fit(X_tr, y_tr).score(X_te, y_te)

# Prediction
predictions = ensemble.predict(X_test)

In [ ]:
out = pd.DataFrame({"Id": test.Id, "SalePrice": predictions})

In [ ]:
out

In [ ]:
out.to_csv("sub.csv", index=False)

In [ ]:
# a Better Score

In [ ]:
train.corr()['SalePrice'].nlargest(10)

In [ ]:
# Outlier
plt.scatter(train['GrLivArea'], train['SalePrice'])

In [ ]:
train = train[train['GrLivArea']<4500]

In [ ]:
plt.scatter(train['GrLivArea'], train['SalePrice'])

In [ ]:
# not Normal Distributed
train['SalePrice'].plot(kind="kde")

In [ ]:
train['SalePrice'] = np.log1p(train['SalePrice'])

In [ ]:
train['SalePrice'].plot(kind="kde")

In [ ]:
train = ds.feature_engineering.drop_missing(train, percent=7.0)

In [ ]:
# Electrical has only one missing
train.isna().sum().sort_values().tail(20)

In [ ]:
train[train['Electrical'].isna()]

In [ ]:
# Remove it
train = train[~train['Electrical'].isna()]

In [ ]:
train[train['MasVnrArea'].isna()]
train[train['MasVnrType'].isna()]

In [ ]:
# drop 37 NaNs
train = train[~train['MasVnrArea'].isna()]
# drop 37 NaNs
train = train[~train['MasVnrType'].isna()]

In [ ]:
# year month to categorical
cols = ["MSSubClass", "YrSold", 'MoSold']
train[cols] = train[cols].astype(str)
test[cols] = test[cols].astype(str)

In [ ]:
train['YrSold']

In [ ]:
# Collinearity
train.corr()['GarageArea']['GarageCars']

In [ ]:
# should remove one

In [ ]:
# not working
# ds.feature_engineering.drop_redundant(train)


# Select upper triangle of correlation matrix
upper = train.corr().abs().where(np.triu(np.ones(train.corr().shape), k=1).astype(np.bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]

In [ ]:
to_drop = to_drop[:4]

In [ ]:
train = train.drop(train[to_drop],axis=1)

In [ ]:
test = test.drop(test[to_drop],axis=1)

In [ ]:
def fill_missing_data(df):
    df_data = df.copy()
    categoricals = []
    for cname,dtype in df_data.dtypes.items():
        if dtype == 'object':
            categoricals.append(cname)
    # Fill 'None' for the Categorical attribute
    df_data[categoricals] = df_data[categoricals].fillna('None')
    
    for cname in df_data.columns:
        if cname not in categoricals:
            df_data[cname] = df_data[cname].fillna(0) #Fill 0 for the Numeric attribute
    return df_data


In [ ]:
train = fill_missing_data(train)
test = fill_missing_data(test)

In [ ]:
train['TotalPorchSF'] = train['OpenPorchSF'] + train['EnclosedPorch'] + train['3SsnPorch'] + train['ScreenPorch']
test['TotalPorchSF'] = test['OpenPorchSF'] + test['EnclosedPorch'] + test['3SsnPorch'] + test['ScreenPorch']

In [ ]:
train['TotalBaths'] = train['BsmtFullBath'] + train['FullBath'] + 0.5*(train['BsmtHalfBath'] + train['HalfBath'])
test['TotalBaths'] = test['BsmtFullBath'] + test['FullBath'] + 0.5*(test['BsmtHalfBath'] + test['HalfBath'])

In [ ]:
train['TotalAreaSF'] = train['TotalBsmtSF'] + train['GrLivArea']
test['TotalAreaSF'] = test['TotalBsmtSF'] + test['GrLivArea']

In [ ]:
train['Age'] = train['YrSold'].astype('int64') - train['YearBuilt']
test['Age'] = test['YrSold'].astype('int64') - test['YearBuilt']

In [ ]:
# one hot encode

In [ ]:
def feature_engineering(df):
    df_data = df.copy()
    
    feature = {
        'categorical':{
            'MSSubClass': ['20', '30', '40', '45', '50', '60', '70', '75', '80', '85', '90', '120', '150', '160', '180', '190'],
            'MSZoning': ['A', 'C', 'FV', 'I', 'RH', 'RL', 'RP', 'RM'],
            'Alley': ['Grvl', 'Pave', 'None'],
            'LandContour': ['Lvl', 'Bnk', 'HLS', 'Low'],
            'LotConfig': ['Inside', 'Corner', 'CulDSac', 'FR2', 'FR3'],
            'Neighborhood': ['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel',
                            'Names', 'NoRidge', 'NPkVill', 'NridgHt', 'NWAmes', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker'],
            'Condition1': ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'],
            'Condition2': ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe'],
            'BldgType': ['1Fam', '2FmCon', 'Duplx', 'TwnhsE', 'TwnhsI'],
            'HouseStyle': ['1Story', '1.5Fin', '1.5Unf', '2Story', '2.5Fin', '2.5Unf', 'SFoyer', 'SLvl'],
            'RoofStyle': ['Flat', 'Gable', 'Gambrel', 'Hip', 'Mansard', 'Shed'],
            'RoofMatl': ['ClyTile', 'CompShg', 'Membran', 'Metal', 'Roll', 'Tar&Grv', 'WdShake', 'WdShngl'],
            'Exterior1st': ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco',
                           'VinylSd', 'Wd Sdng', 'WdShing'],
            'Exterior2nd': ['AsbShng', 'AsphShn', 'BrkComm', 'BrkFace', 'CBlock', 'CemntBd', 'HdBoard', 'ImStucc', 'MetalSd', 'Other', 'Plywood', 'PreCast', 'Stone', 'Stucco',
                           'VinylSd', 'Wd Sdng', 'WdShing'],
            'MasVnrType': ['BrkCmn', 'BrkFace', 'CBlock', 'None', 'Stone'],
            'Foundation': ['BrkTil', 'CBlock', 'PConc', 'Slab', 'Stone', 'Wood'],
            'Heating': ['Floor', 'GasA', 'GasW', 'Grav', 'OthW', 'Wall'],
            'Electrical': ['SBrkr', 'FuseA', 'FuseF', 'FuseP', 'Mix'],
            'Functional': ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'],
            'GarageType': ['2Types', 'Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd', 'None'],
            'GarageFinish': ['Fin', 'RFn', 'Unf', 'None'],
            'PavedDrive': ['Y', 'P', 'N'],
            'MiscFeature': ['Elev', 'Gar2', 'Othr', 'Shed', 'TenC', 'None'],
            'MoSold': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
            'YrSold': ['2006', '2007', '2008', '2009', '2010'],
            'SaleType': ['WD', 'CWD', 'VWD', 'New', 'COD', 'Con', 'ConLw', 'ConLI', 'ConLD', 'Oth'],
            'SaleCondition': ['Normal', 'Abnorml', 'AdjLand', 'Alloca', 'Family', 'Partial']
        },
        'binary': {
            'Street': ['Pave', 'Grvl'],
            'CentralAir': ['Y', 'N']          
        },
        'ordinal': {
            'LotShape': ['None', 'IR3', 'IR2', 'IR1', 'Reg'],
            'Utilities': ['None', 'NoSeWa', 'NoSewr', 'AllPub'],
            'LandSlope': ['None', 'Sev', 'Mod', 'Gtl'],
            'ExterQual': ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
            'ExterCond': ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
            'BsmtQual': ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
            'BsmtCond': ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
            'BsmtExposure': ['None', 'No', 'Mn', 'Av', 'Gd'],
            'BsmtFinType1': ['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
            'BsmtFinType2': ['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
            'HeatingQC': ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
            'KitchenQual': ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
            'FireplaceQu': ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
            'GarageQual': ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
            'GarageCond': ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
            'Fence': ['None', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv'],
            'PoolQC': ['None', 'Fa', 'Ta', 'Gd', 'Ex']
        },
    }
    
    selected = []
    for cname in df_data.columns:
        if cname in feature['binary']: # Convert the binary attributes to 0/1
            default_value = feature['binary'][cname][0]
            feature_name = cname + "_is_" + default_value
            selected.append(feature_name)
            df_data[feature_name] = df_data[cname].apply(lambda x: int(x == default_value))
        elif cname in feature['categorical']: # Convert Categorical attributes into One-hot vector
            values = feature['categorical'][cname]
            for val in values:
                try:
                    new_name = "{}_{}".format(cname, val)

                    selected.append(new_name)
                    df_data[new_name] = df_data[cname].apply(lambda x: int(x == val))
                except Exception as err:
                    print("One-hot encoding for {}_{}. Error: {}".format(cname, val, err))
        elif cname in feature['ordinal']: # Convert the Ordinal attributes to a number
            new_name = cname + "_ordinal"
            selected.append(new_name)
            df_data[new_name] = df_data[cname].apply(lambda x: int(feature['ordinal'][cname].index(x)))
        else: # The remaining attributes are numeric so they remain the same
#             print(cname)
            selected.append(cname)
            
    return df_data[selected]

In [ ]:
train = feature_engineering(train)
test = feature_engineering(test)

In [ ]:
train

In [ ]:
# remove all zeros columns
# df=df[[i for i in df if len(set(df[i]))>1]]
for col in train.columns:    
    if any(train[col]) == False:
            train.drop([col], axis=1, inplace=True)
            test.drop([col], axis=1, inplace=True)

In [ ]:
id = test['Id']

In [ ]:
id.shape

In [ ]:
y = train['SalePrice']

In [ ]:
y.shape

In [ ]:
test['Id'].shape

In [ ]:
df['SalePrice']

In [ ]:
train.drop(['Id', 'SalePrice'], axis=1, inplace=True)

test.drop(['Id'], axis=1, inplace=True)

In [ ]:
abt = test

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train = scaler.fit_transform(train)
# test = scaler.transform(test)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.25, random_state=1)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=123)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
param_init = {
    "max_depth": 5, # default: 3 only for depthwise
    "n_estimators": 3000, # default: 500
    "learning_rate": 0.01, # default: 0.05
    "subsample": 0.5,
    "colsample_bytree": 0.7,  # default:  1.0
    "min_child_weight": 1.5,
    "reg_alpha": 0.75,
    "reg_lambda": 0.4,
    "seed": 42,
#     "eval_metric": "rmse"
}

In [ ]:
import xgboost
xgb_model = xgboost.XGBRegressor(**param_init)

In [ ]:
param_fit = {
    "eval_metric": "rmse",
    "early_stopping_rounds": 500, # default: 100
    "verbose": 200,
    "eval_set": [(X_val, y_val)]
}

In [ ]:
xgb_model = xgb_model.fit(X_train, y_train, **param_fit)


In [ ]:
y_pred_xgb = xgb_model.predict(X_test)

In [ ]:
SalePrice_pred

In [ ]:
mean_squared_error(y_test, y_pred_xgb, squared=False)


In [ ]:
SalePrice_pred = np.exp(y_pred_xgb)

In [ ]:
y_pred_xgb.shape

In [ ]:
# Id=pd.Series(range(1461,2920))

In [ ]:
# submission = pd.DataFrame({'Id': Id, 'SalePrice': SalePrice_pred})

In [ ]:
# submission.to_csv("sub.csv", index=False)